In [4]:
import numpy as np
import pandas as pd
import cv2
import os
from keras import layers
from keras.layers import Input, Dense, Activation, Flatten, Conv2D
from keras.layers import MaxPooling2D
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K
# imageSize= 50

In [5]:
totalnum_of_images = 2648
datadir = "/Users/sarveshbhatnagar/Desktop/AJIP/project_OCA"
categories = {0:"fan_0", 1:"fan_1", 2:"fan_2", 3:"fan_3"}
counter = 0
dataset = np.zeros(shape=(totalnum_of_images,65537))

for category in categories:
    path = os.path.join(datadir,categories[category])
    label = np.array([category])
    for img in os.listdir(path):
        try:
            image = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (256,256))
            te = image.flatten()
            te = np.array(te)
            te = np.concatenate((te, label), axis=0)
            te = np.array(te)[np.newaxis]
            #cv2.imwrite("newfan"+str(i)+".jpg",image)
            dataset[counter] = te
            counter = counter + 1
        except Exception as e:
            pass

print(dataset.shape)

(2648, 65537)


In [6]:
dataset[1]

array([169., 169., 170., ..., 129., 129.,   0.])

In [7]:
np.random.shuffle(dataset)
X = dataset
Y = dataset
#sclicing only the first 1023 columns so that we would get our features
x = X[:, 0:65536]
#sclicing the 1024th column which has the label for every record
y = Y[:, 65536]
#as the dataset is shuffled so selecting first 70K tuples won't harm
x_train = x[0:2118, :]
#diving by 255 makes it easier for learning data as now the value is in between 0 and 1(basically normalization)
x_train = x_train/255

x_test = x[2118:2648, :]
#adjusting the shape of the matrices
y = y.reshape(y.shape[0], 1)
y_train = y[0:2118, :]
y_train = y_train.T
y_test = y[2118:2648, :]
y_test = y_test.T

In [8]:
#we are taking an image of 32x32 for training purpose
image_x = 256
image_y = 256
#to_categorical is an function which converts a matrix to a vector; in our case it basically gets the label which is an integer
#and changes it to form of one_hot_encoding.
#Check this link for more details->https://keras.io/utils/
train_y = np_utils.to_categorical(y_train, dtype='int32')
test_y = np_utils.to_categorical(y_test, dtype='int32')
#adjusting the shape
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
x_train = x_train.reshape(x_train.shape[0], image_x, image_y, 1)
x_test = x_test.reshape(x_test.shape[0], image_x, image_y, 1)

In [9]:
#explained in Readme file
def keras_model(image_x, image_y):
    num_of_classes = 4
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(10,10), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    model.add(Conv2D(64, (10,10), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #saving the trained data is saved here
    filepath='OCA_1.h5'
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]
    #returning the model and the checkpoints of the saved(learned) data
    return model, callbacks_list

In [10]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(x_train, train_y, validation_data=(x_test, test_y), epochs=10, batch_size=64, callbacks=callbacks_list)
scores = model.evaluate(x_test, test_y, verbose=0)
print("CNN error: %.2f%%" %(100 - scores[1] * 100))
print_summary(model)
model.save('OCA_1.h5')

Train on 2118 samples, validate on 530 samples
Epoch 1/10
2118/2118 [==============================] - 281s 133ms/step - loss: 1.3095 - acc: 0.3942 - val_loss: 9.6359 - val_acc: 0.3962

Epoch 00001: val_acc improved from -inf to 0.39623, saving model to OCA_1.h5
Epoch 2/10
2118/2118 [==============================] - 269s 127ms/step - loss: 1.0161 - acc: 0.5732 - val_loss: 6.1373 - val_acc: 0.5906

Epoch 00002: val_acc improved from 0.39623 to 0.59057, saving model to OCA_1.h5
Epoch 3/10
2118/2118 [==============================] - 274s 129ms/step - loss: 0.7711 - acc: 0.6955 - val_loss: 5.5004 - val_acc: 0.6547

Epoch 00003: val_acc improved from 0.59057 to 0.65472, saving model to OCA_1.h5
Epoch 4/10
2118/2118 [==============================] - 1414s 668ms/step - loss: 0.5943 - acc: 0.7786 - val_loss: 6.0396 - val_acc: 0.6226

Epoch 00004: val_acc did not improve from 0.65472
Epoch 5/10
2118/2118 [==============================] - 273s 129ms/step - loss: 0.5149 - acc: 0.7932 - val_lo

In [11]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [13]:
import tensorflow as tf
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, "pbmodel", "my_model.pb", as_text=False)

INFO:tensorflow:Froze 29 variables.
INFO:tensorflow:Converted 29 variables to const ops.


'pbmodel/my_model.pb'